# Gradio app with no map

This project is a demo of MCP (Model Context Protocol) capabilities. MCP has made waves in the data science community lately with the rise of agentic LLMs. Recently, [Gradio announced](https://www.gradio.app/guides/building-mcp-server-with-gradio) that they would support MCP natively in their applications. This one-line change is a game changer for developers exploring this capability. 

Could not have done this without the contributions by the [ADS-B Exchange Community](https://www.adsbexchange.com/). I accesses the data through the [RapidAPI](https://rapidapi.com/adsbx/api/adsbexchange-com1). 

### Import necessary libraries.

In [13]:
import gradio as gr
import requests
import os
from dotenv import load_dotenv

### Setup the API creds. 

In [14]:
# Load your API key from a .env file
load_dotenv()
RAPIDAPI_KEY = os.getenv("RAPIDAPI_KEY")

### Function to grab aircraft within 5 NM of location. 

In [15]:
# Function to query the ADS-B Exchange API
def find_aircraft(lat, lon):
    """
    Query the ADS-B Exchange API to find aircraft within 1 nautical mile of a given latitude and longitude.

    Parameters
    ----------
    lat : float
        The latitude in decimal degrees where the aircraft search should be centered.
    lon : float
        The longitude in decimal degrees where the aircraft search should be centered.

    Returns
    -------
    str
        A string describing the number of aircraft found and their callsigns and types,
        or a message indicating that no aircraft were found or that an error occurred.
    """
    try:
        
        url = f"https://adsbexchange-com1.p.rapidapi.com/v2/lat/{lat}/lon/{lon}/dist/5/"

        headers = {
            "x-rapidapi-key": RAPIDAPI_KEY,
            "x-rapidapi-host": "adsbexchange-com1.p.rapidapi.com",
        }

        response = requests.get(url, headers=headers)
        data = response.json()

        if "ac" in data and data["ac"]:
            aircraft_info = [
                f"Flight: {ac.get('flight', 'N/A')} | Type: {ac.get('t', 'N/A')}"
                for ac in data["ac"]
            ]
            return f"{len(data['ac'])} aircraft found:\n\n" + "\n".join(aircraft_info)
        else:
            return "No aircraft found within 5 NM."

    except Exception as e:
        return f"Error: {str(e)}"

### Setup the Gradio interface. 

In [16]:
# Gradio UI
demo = gr.Interface(
    fn=find_aircraft,
    inputs=[
        gr.Number(label="Latitude", value=27.941042),
        gr.Number(label="Longitude", value=-82.455581),
    ],
    outputs="text",
    title="Nearby Aircraft Finder (ADS-B Exchange)",
    description="Enter a latitude and longitude to find aircraft within 5 nautical mile using ADS-B Exchange's REST API.",
)

### Finally, call the Gradio UI. 

In [18]:
demo.launch(mcp_server=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
* To create a public link, set `share=True` in `launch()`.

🔨 MCP server (using SSE) running at: http://127.0.0.1:7863/gradio_api/mcp/sse
